In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from sklearn.datasets import fetch_20newsgroups
categories = ['alt.atheism', 'talk.religion.misc','comp.graphics', 'sci.space']
categories = None
df = fetch_20newsgroups(categories=categories, shuffle=True, random_state=42)

from pprint import pprint
pprint(list(df.target_names))

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']


In [3]:
df.filenames.shape

(11314,)

In [4]:
df.target.shape

(11314,)

In [5]:
# Convert to list
data = df.data

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

<input>:5: DeprecationWarning: invalid escape sequence \S
<input>:8: DeprecationWarning: invalid escape sequence \s
<input>:5: DeprecationWarning: invalid escape sequence \S
<input>:8: DeprecationWarning: invalid escape sequence \s
<input>:5: DeprecationWarning: invalid escape sequence \S
<input>:8: DeprecationWarning: invalid escape sequence \s
<ipython-input-5-1eed9f39b362>:5: DeprecationWarning: invalid escape sequence \S
  data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
<ipython-input-5-1eed9f39b362>:8: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', sent) for sent in data]


['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


In [6]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


In [7]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


In [8]:
# NLTK Stop words
import nltk; nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asergeev\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['where', 's', 'thing', 'car', 'nntp_post', 'host', 'rac_wam', 'umd', 'organization', 'university', 'maryland_college', 'park', 'line', 'wonder', 'anyone', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'front_bumper', 'separate', 'rest', 'body', 'know', 'anyone', 'tellme', 'model', 'name', 'engine', 'spec', 'year', 'production', 'car', 'make', 'history', 'whatev', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


In [10]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 5), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 2), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1)]]


In [11]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('addition', 1),
  ('anyone', 2),
  ('body', 1),
  ('bricklin', 1),
  ('bring', 1),
  ('call', 1),
  ('car', 5),
  ('could', 1),
  ('day', 1),
  ('door', 2),
  ('early', 1),
  ('engine', 1),
  ('enlighten', 1),
  ('front_bumper', 1),
  ('funky', 1),
  ('history', 1),
  ('host', 1),
  ('info', 1),
  ('know', 1),
  ('late', 1),
  ('lerxst', 1),
  ('line', 1),
  ('look', 2),
  ('mail', 1),
  ('make', 1),
  ('maryland_college', 1),
  ('model', 1),
  ('name', 1),
  ('neighborhood', 1),
  ('nntp_post', 1),
  ('organization', 1),
  ('park', 1),
  ('production', 1),
  ('rac_wam', 1),
  ('really', 1),
  ('rest', 1),
  ('s', 1),
  ('see', 1),
  ('separate', 1),
  ('small', 1),
  ('spec', 1),
  ('sport', 1),
  ('tellme', 1),
  ('thank', 1),
  ('thing', 1),
  ('umd', 1),
  ('university', 1),
  ('whatev', 1),
  ('where', 1),
  ('wonder', 1),
  ('year', 1)]]

In [12]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [13]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.153*"window" + 0.065*"image" + 0.045*"display" + 0.044*"server" + '
  '0.037*"screen" + 0.020*"directory" + 0.020*"run" + 0.020*"font" + '
  '0.017*"cub" + 0.016*"object"'),
 (1,
  '0.053*"armenian" + 0.041*"greek" + 0.033*"turk" + 0.027*"turkish" + '
  '0.021*"turkey" + 0.018*"russian" + 0.015*"gif" + 0.014*"serdar_argic" + '
  '0.013*"armenia" + 0.013*"closed"'),
 (2,
  '0.042*"not" + 0.030*"would" + 0.027*"do" + 0.021*"be" + 0.016*"get" + '
  '0.015*"go" + 0.015*"think" + 0.014*"make" + 0.014*"good" + 0.014*"know"'),
 (3,
  '0.095*"sale" + 0.043*"tape" + 0.042*"character" + 0.030*"price" + '
  '0.029*"sell" + 0.023*"chris" + 0.023*"offer" + 0.021*"rider" + '
  '0.017*"jason" + 0.015*"decent"'),
 (4,
  '0.069*"color" + 0.059*"test" + 0.058*"video" + 0.049*"monitor" + '
  '0.046*"scan" + 0.039*"mouse" + 0.029*"mhz" + 0.020*"mirror" + '
  '0.020*"battery" + 0.018*"convert"'),
 (5,
  '0.049*"bike" + 0.028*"eat" + 0.028*"kid" + 0.026*"air" + 0.021*"dod" + '
  '0.021*"doctor" + 

In [14]:
gamma, _ = lda_model.inference(corpus)
doc_topic_dists = gamma / gamma.sum(axis=1)[:, None]

In [15]:
doc_topic_dists

array([[2.2523028e-03, 9.7823690e-04, 2.6331052e-01, ..., 5.1088899e-02,
        2.2077983e-02, 1.6128867e-03],
       [1.8143454e-03, 7.8801910e-04, 1.4605649e-01, ..., 3.9154660e-02,
        2.2608619e-02, 1.2992630e-03],
       [2.3491610e-02, 3.9957426e-04, 4.2423522e-01, ..., 4.9117520e-02,
        3.2734085e-02, 6.5880601e-04],
       ...,
       [2.3967884e-03, 1.0409901e-03, 3.3047873e-01, ..., 3.9565694e-02,
        1.6842678e-02, 1.7163537e-03],
       [1.9569714e-03, 8.4996544e-04, 3.5768721e-01, ..., 2.5382737e-02,
        4.8982359e-02, 1.4014564e-03],
       [2.3804752e-02, 1.3431321e-03, 2.3406911e-01, ..., 4.0973231e-02,
        4.1263252e-02, 2.2145172e-03]], dtype=float32)

In [16]:
doc_topic_dists.shape

(11314, 20)

In [42]:
from sklearn.metrics import pairwise_distances
dist = pairwise_distances(doc_topic_dists)
np.fill_diagonal(dist, float("inf"))
dist

D:\Anaconda\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)


array([[       inf, 0.33356097, 0.29926068, ..., 0.2918661 , 0.22259098,
        0.16839483],
       [0.33356097,        inf, 0.39716285, ..., 0.2937796 , 0.3333894 ,
        0.29101664],
       [0.29926068, 0.39716285,        inf, ..., 0.22098449, 0.2200688 ,
        0.23893002],
       ...,
       [0.2918661 , 0.2937796 , 0.22098449, ...,        inf, 0.27570096,
        0.2575394 ],
       [0.22259098, 0.3333894 , 0.2200688 , ..., 0.27570096,        inf,
        0.1890872 ],
       [0.16839483, 0.29101664, 0.23893002, ..., 0.2575394 , 0.1890872 ,
               inf]], dtype=float32)

In [43]:
dist.shape

(11314, 11314)

In [44]:
score = 0

for i in range(len(dist)):
    indx = np.argsort(dist[i][:101])
    score += sum(df.target[indx] == df.target[i]) /100

In [45]:
score / len(dist)

0.05149195686759611

In [19]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

D:\Anaconda\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.365028 -0.042478       1        1  32.994583
10    -0.290486  0.107284       2        1  14.937504
6     -0.258537  0.209936       3        1  11.932644
18    -0.277515 -0.172091       4        1   8.774340
8     -0.135550 -0.101573       5        1   5.352482
17    -0.127677 -0.020703       6        1   5.193089
9      0.050794  0.016789       7        1   4.724924
15    -0.062915 -0.180926       8        1   3.910157
13     0.051748  0.159981       9        1   1.842016
14     0.030251  0.220712      10        1   1.711360
7      0.106603  0.007598      11        1   1.180165
5      0.129223 -0.007934      12        1   1.104178
0      0.120879  0.035351      13        1   1.071586
1      0.106939 -0.093650      14        1   1.053303
19     0.093704 -0.107670      15        1   1.044910
12     0.168106 -0.008354      16        1   0.772133
4      0.154133 -0.013843      17        1   0.728299
11     0.161848 -0.009031      18        1   0.720527
3      0.151401  0.012156      19        1   0.684873
16     0.192077 -0.011557      20        1   0.266924, topic_info=     Category          Freq            Term         Total  loglift  logprob
term                                                                       
5615  Default  50327.000000              ax  50327.000000  30.0000  30.0000
21    Default  18448.000000            line  18448.000000  29.0000  29.0000
30    Default  16928.000000    organization  16928.000000  28.0000  28.0000
82    Default  22603.000000             not  22603.000000  27.0000  27.0000
1120  Default   4799.000000               _   4799.000000  26.0000  26.0000
245   Default  14270.000000           write  14270.000000  25.0000  25.0000
55    Default  10841.000000         article  10841.000000  24.0000  24.0000
65    Default  14492.000000              do  14492.000000  23.0000  23.0000
29    Default   6795.000000       nntp_post   6795.000000  22.0000  22.0000
46    Default   6844.000000      university   6844.000000  21.0000  21.0000
210   Default  17293.000000           would  17293.000000  20.0000  20.0000
16    Default   6342.000000            host   6342.000000  19.0000  19.0000
2450  Default   4027.000000             max   4027.000000  18.0000  18.0000
50    Default   5394.000000            year   5394.000000  17.0000  17.0000
1394  Default   2638.000000          window   2638.000000  16.0000  16.0000
166   Default   8109.000000          people   8109.000000  15.0000  15.0000
111   Default  11037.000000              be  11037.000000  14.0000  14.0000
435   Default   3287.000000           drive   3287.000000  13.0000  13.0000
401   Default   2261.000000            file   2261.000000  12.0000  12.0000
368   Default   9094.000000             say   9094.000000  11.0000  11.0000
241   Default   5728.000000          system   5728.000000  10.0000  10.0000
835   Default   3200.000000             key   3200.000000   9.0000   9.0000
134   Default  10799.000000             get  10799.000000   8.0000   8.0000
651   Default   3665.000000             god   3665.000000   7.0000   7.0000
916   Default   1930.000000           space   1930.000000   6.0000   6.0000
6     Default   2110.000000             car   2110.000000   5.0000   5.0000
377   Default   3672.000000           state   3672.000000   4.0000   4.0000
409   Default   4078.000000           reply   4078.000000   3.0000   3.0000
274   Default   4133.000000           right   4133.000000   2.0000   2.0000
930   Default   2772.000000            team   2772.000000   1.0000   1.0000
...       ...           ...             ...           ...      ...      ...
3860  Topic20    178.681656        michigan    180.035660   5.9184  -3.1819
1385  Topic20    148.371780         session    149.725784   5.9169  -3.3678
2828  Topic20    122.340103       astronomy    123.694107   5.9150  -3.5607
1375  Topic20     90.740471  impl

In [21]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -14.222266569674229

Coherence Score:  0.5262293975319933
